In [2]:
import pandas as pd
from pathlib import Path
from typing import List
from typing import Dict
from pprint import pprint
import pickle
import os
from os import path
from collections import Counter
import datetime

### Составление списка всех названий колонок из всех файлов: группировка всех одинаковых по смыслу и разных по написанию колонок

In [2]:
path_to_data = Path('/s/ls4/groups/g0126/datasets/Drilling_data/Geonavigation/wells/')


In [3]:
start = datetime.datetime.now()
print('Время старта: ' + str(start))

column_names = []

for fp in path_to_data.iterdir():
    df_well = pd.read_csv(fp, index_col=0, sep=';')
    
    # Добавляем в спиоск названия колонок
    column_names.extend(df_well.columns.tolist())

#Убираем дубликаты 

column_names = list(set(column_names))
print(len(column_names))

finish = datetime.datetime.now()
print('Время окончания: ' + str(finish))
print('Время работы: ' + str(finish - start))

Время старта: 2024-10-20 22:22:25.692568
93
Время окончания: 2024-10-20 22:24:41.775669
Время работы: 0:02:16.083101


In [4]:
print(column_names)

['Расход р-ра выход (%) (%)', 'Ходы насоса 1 (spm)', 'KPI states (euc)', 'Расход раствора вход (L/min)', 'Верт.глубина долота (m)', 'Мех. скорость (m/h)', 'Уд.вес раствора вход (kgm3)', 'Расход р-ра вход (l/m)', 'Забой (m)', 'Объём раствора в емкости 8 (m3)', 'Объём раствора в емкости 3 (m3)', 'Расход р-ра выход (l/m)', 'Расход выход (l/m)', 'Давление (kPa)', 'Раствор вход (L/m)', 'Положение долота (m)', 'Давление (kpa)', 'Расход р-ра вход (L/min)', 'Объём раствора в емкости 5 (m3)', 'Объем раствора (m3)', 'Вес на крюке (макс) (kdn)', 'OpKPI ()', 'DEPTBITM (m)', 'Расход вход (L/min)', 'Расход раствора выход (%) (%)', 'Расход р-ра выход (L/min)', 'Секции ()', 'Объём раствора в емкости 10 (m3)', 'Объём раствора в емкости 7 (m3)', 'Ходы насоса (общ.) (euc)', 'Проводимость раствора выход (mmho)', 'KPI ()', 'Вес на крюке (t)', 'Расход раствора выход (l/m)', 'HKLD (1000 lbf)', 'Расход раствора выход (L/min)', 'Объём раствора в емкости 9 (m3)', 'Температура раствора выход (degc)', 'GRM1 (gAPI

In [53]:
# словарь, где ключи: колонки, которые надо изменить, значения: на что надо изменить

fix_columns_dict = {
  "Вес на крюке (10 kN)": "Вес на крюке (t)", # 10 Килоньютон (Масса) = 1.0197 Тонн
  "HKLD (1000 lbf)": "Вес на крюке (t)",  # 1000 фунтов (lbf) равны 0,453592 тонны. 
  
  "Давление (psi)": "Давление (kpa)",   # 1 psi = 6,89475729 килопаскалей (кПа).
  "Давление (kPa)": "Давление (kpa)", 
  "SPPA (psi)": "Давление (kpa)",   # 1 psi [psi] = 6,89475729317831 килопаскаль [кПа]

  #"Забой (метры) ()": "Забой (m)",
  
  "Мех. скорость (m/hr)": "Мех. скорость (m/h)",
    
  "Момент (kN.m)": "Момент (knm)",
  "Момент (макс) (kN.m)": "Момент (макс) (knm)",

  "Нагрузка на долото (10 kN)": "Нагрузка на долото (t)",   # 10 Килоньютон (Масса) = 1.0197 Тонн
  "Нагрузка на долото (макс) (kdn)": "Нагрузка на долото (макс) (t)", # Вопрос с kdn
  
  #"Положение долота (метры) ()": "Положение долота (m)",
  #"DEPTBITM (m)": "Положение долота (m)",

  #"Забой (метры) ()": "Забой (m)",
  
  "Расход вход (L/min)": "Расход вход (l/m)",
  "Расход р-ра вход (l/m)": "Расход вход (l/m)",
  "Расход р-ра вход (L/min)": "Расход вход (l/m)",
  "Расход раствора вход (l/m)": "Расход вход (l/m)",
  "Расход раствора вход (L/min)": "Расход вход (l/m)",
  
  "Расход выход (L/min)": "Расход выход (l/m)",
  "Расход р-ра выход (l/m)": "Расход выход (l/m)",
  "Расход р-ра выход (L/min)": "Расход выход (l/m)",
  "Расход раствора выход (l/m)": "Расход выход (l/m)",
  "Расход раствора выход (L/min)": "Расход выход (l/m)",
  
  "Расход р-ра выход (%) (%)": "Расход выход (%) (%)",
  "Расход раствора выход (%) (%)": "Расход выход (%) (%)",

  "Скорость блока (ft/s)": "Скорость блока (m/s)", # 1 t/s =  0,3048 m/s
  "Скорость блока ()": "Скорость блока (m/s)",
  "Скорость блока Abs (m/s)": "Скорость блока (Абс) (m/s)",

  #"Тальблок (метры) (m)": "Тальблок (m)",
  "Operations (euc)": "Операции (euc)"
}


conversion_factors = {
    "Вес на крюке (10 kN)": 1.0197,  # 10 kN в тонны
    "HKLD (1000 lbf)": 0.453592,      # 1000 lbf в тонны
    "Давление (psi)": 6.89475729,     # psi в кПа
    "SPPA (psi)": 6.89475729,         # psi в кПа
    "Нагрузка на долото (10 kN)": 1.0197,  # 10 kN в тонны
    "Скорость блока (ft/s)": 0.3048      # ft/s в m/s
}

### Создание колонок abscent, nan%>=0.3, std, zero%>0.8, most_popular%>=0.8, sum

In [8]:
import pandas as pd
from collections import Counter
from pathlib import Path
import datetime

start = datetime.datetime.now()
print('Время старта: ' + str(start))

path_to_data = Path('/s/ls4/groups/g0126/datasets/Drilling_data/Geonavigation/wells/')

# Переименование колонок в списке с использованием словаря
renamed_columns = list(set([fix_columns_dict.get(name, name) for name in column_names]))

new_table = pd.DataFrame({'Parameters': renamed_columns})

# Инициализация столбцов
for col_name in ['abscent', 'nan%>=0.3', 'std', 'zero%>0.8', 'most_popular%>=0.8']:
    new_table[col_name] = 0

# Подсчет параметров
#param_counts = Counter(renamed_columns)

# Обработка файлов
for fp in path_to_data.iterdir():
    
    df_i = pd.read_csv(fp, index_col=0, sep=';')

    for column, factor in conversion_factors.items():
            if column in df_i.columns: 
                df_i[column] *= factor  # Домножаем на коэффициент
                
    
    # Переименование колонок в исходных таблицах
    df_i.rename(columns=fix_columns_dict, inplace=True)
    
    for param in new_table['Parameters']:
        
        if param not in df_i.columns:
            new_table.loc[new_table['Parameters'] == param, 'abscent'] += 1 
            
        if param in df_i.columns:
            if not df_i[param].empty:
                
                nan_percent = df_i[param].isnull().mean() * 100
                
                if isinstance(nan_percent, (float, int)) and nan_percent >= 30.0:
                    new_table.loc[new_table['Parameters'] == param, 'nan%>=0.3'] += 1
                
                if pd.api.types.is_numeric_dtype(df_i[param]):  
                    std_val = df_i[param].std()

                    if isinstance(std_val, (float, int)) and std_val == 0:
                        new_table.loc[new_table['Parameters'] == param, 'std'] += 1

                if pd.api.types.is_numeric_dtype(df_i[param]):  
                    zero_percent = (df_i[param] == 0).mean() * 100
                    
                    if isinstance(zero_percent, (float, int)) and zero_percent > 80:
                        new_table.loc[new_table['Parameters'] == param, 'zero%>0.8'] += 1

                most_common_count = Counter(df_i[param]).most_common(1)
                if most_common_count: 
                    
                    total_count = len(df_i[param])
                    
                    if total_count > 0:
                        most_common_percent = most_common_count[0][1] / total_count * 100
                        
                        if isinstance(most_common_percent, (float, int)) and most_common_percent >= 80:
                            new_table.loc[new_table['Parameters'] == param, 'most_popular%>=0.8'] += 1




new_table.set_index('Parameters', inplace=True)

new_table['sum'] = new_table.sum(axis=1)


finish = datetime.datetime.now()
print('Время окончания: ' + str(finish))
print('Время работы: ' + str(finish - start))


Время старта: 2024-10-20 23:19:36.967357
Время окончания: 2024-10-20 23:26:55.458264
Время работы: 0:07:18.490907


In [9]:
new_table

,abscent,nan%>=0.3,std,zero%>0.8,most_popular%>=0.8,sum
Parameters,,,,,,
Тальблок (метры) (m),25,0,0,0,0,25
Объем раствора (m3),0,0,0,0,0,0
Тальблок (m),0,0,0,0,0,0
Ходы насоса 1 (spm),0,0,0,4,4,8
Нагрузка на долото (макс) (t),9,0,0,4,4,17
...,...,...,...,...,...,...
Раствор выход (L/m),10,7,0,1,1,19
Объём доливной ёмкости 1 (m3),25,1,1,0,0,27
KPIsCorr (),0,1,0,0,0,1


In [10]:
flag = new_table
flag

,abscent,nan%>=0.3,std,zero%>0.8,most_popular%>=0.8,sum
Parameters,,,,,,
Тальблок (метры) (m),25,0,0,0,0,25
Объем раствора (m3),0,0,0,0,0,0
Тальблок (m),0,0,0,0,0,0
Ходы насоса 1 (spm),0,0,0,4,4,8
Нагрузка на долото (макс) (t),9,0,0,4,4,17
...,...,...,...,...,...,...
Раствор выход (L/m),10,7,0,1,1,19
Объём доливной ёмкости 1 (m3),25,1,1,0,0,27
KPIsCorr (),0,1,0,0,0,1


In [11]:
# Сортировка по сумме
result = flag.loc[flag['abscent'] == 0].sort_values(by='sum')
result = result.reset_index()
result.to_csv('/s/ls4/users/selishcheva/VKR/statistics_table.csv', index = True)
result

,Parameters,abscent,nan%>=0.3,std,zero%>0.8,most_popular%>=0.8,sum
0,Объем раствора (m3),0,0,0,0,0,0
1,Положение долота (m),0,0,0,0,0,0
2,Вес на крюке (t),0,0,0,0,0,0
3,Забой (m),0,0,0,0,0,0
4,Секции (),0,0,0,0,0,0
5,Операции (euc),0,0,0,0,0,0
6,Мех. скорость (m/h),0,0,0,0,0,0
7,KPI states (euc),0,0,0,0,0,0
8,Тальблок (m),0,0,0,0,0,0
9,OperCorr (),0,0,0,0,0,0


In [ ]:
# Колонки, которые были в ВКР у Соболева 

sobol_columns = ["Нагрузка на долото (t)", "Вес на крюке (t)", "Мех. скорость (m/h)", "Скорость блока (m/s)", 
                 "Давление (kpa)", "Тальблок (m)", "Забой (m)", "Положение долота (m)"], "Вес на крюке (t)"]

# Также была колонка "Разница между значением потока БР на выходе и входе баррель нефтяной США числовой"

### Итоговое преобразование данных

In [4]:
result = pd.read_csv('/s/ls4/users/selishcheva/VKR/statistics_table.csv')
result

,Unnamed: 0,Parameters,abscent,nan%>=0.3,std,zero%>0.8,most_popular%>=0.8,sum
0,0,Объем раствора (m3),0,0,0,0,0,0
1,1,Положение долота (m),0,0,0,0,0,0
2,2,Вес на крюке (t),0,0,0,0,0,0
3,3,Забой (m),0,0,0,0,0,0
4,4,Секции (),0,0,0,0,0,0
5,5,Операции (euc),0,0,0,0,0,0
6,6,Мех. скорость (m/h),0,0,0,0,0,0
7,7,KPI states (euc),0,0,0,0,0,0
8,8,Тальблок (m),0,0,0,0,0,0
9,9,OperCorr (),0,0,0,0,0,0


In [10]:
start = datetime.datetime.now()
print('Время старта: ' + str(start))

path_to_data = Path('/s/ls4/groups/g0126/datasets/Drilling_data/Geonavigation/wells/')

parameters_set = set(result['Parameters'].unique())
lims = pd.read_csv('param_limits.csv')

for fp in path_to_data.iterdir():
    
    #if fp.is_file():
    
    df_i = pd.read_csv(fp, index_col=0, sep=';')

    # Приводим к единой размерности
    for column, factor in conversion_factors.items():
        if column in df_i.columns: 
            df_i[column] *= factor  # Домножаем на коэффициент
                
    
    # Переименование колонок в исходных таблицах
    df_i.rename(columns=fix_columns_dict, inplace=True)

    # Удаление тех колонок, которые не соответствуют требованиям по табличке
    df_i.drop(columns=[col for col in df_i.columns if col not in parameters_set], inplace=True)

    df_i.drop(columns=['OperCorr ()', 'KPIsCorr ()', 'Секции ()', 'KPI states (euc)'], inplace=True)

    # Заполнение предыдущими значениями ячеек, у которых значения параметров выходят за пределы лимитов
    for param in df_i.columns:
    
        if param in lims.columns:
        
             min_val = lims.loc[lims['name'] == param, 'min_val'].values[0]
             max_val = lims.loc[lims['name'] == param, 'max_val'].values[0]
             df_i[param] = df_i[param].mask(df_i[param] < min_val, other=pd.NA)
             df_i[param] = df_i[param].mask(df_i[param] > max_val, other=pd.NA)

    
    df_i.ffill(inplace=True)
    
    df_i.dropna(inplace=True)

    df_i = df_i[~df_i.index.duplicated(keep='first')]
        
    df_i.index = pd.to_datetime(df_i.index)
    
    # Переиндексируем с шагом в 10 секунд
    new_index = pd.date_range(start=df_i.index.min(), end=df_i.index.max(), freq='10s')
    
    df_i = df_i.reindex(new_index, method='ffill')
    
    df_i.to_csv(f'/s/ls4/users/selishcheva/VKR/well_data/{fp.name}')


finish = datetime.datetime.now()
print('Время окончания: ' + str(finish))

print('Время работы: ' + str(finish - start))


Время старта: 2024-10-28 12:30:25.590124


FileNotFoundError: [Errno 2] No such file or directory: '/s/ls4/groups/g0126/datasets/Drilling_data/Geonavigation/wells'

In [14]:
import pandas as pd
df_well_ch = pd.read_csv('/s/ls4/users/selishcheva/VKR/well_data/well_02_01_001.csv', index_col = 0)

In [15]:
df_well_ch

,Операции (euc),Вес на крюке (t),Газ (%),Давление (kpa),Забой (m),Мех. скорость (m/h),Момент (knm),Нагрузка на долото (t),Обороты ротора (rpm),Объем раствора (m3),Положение долота (m),Тальблок (m),Ходы насоса 1 (spm),Ходы насоса 2 (spm)
2019-12-26 16:01:04,9.0,1.54,0.04,0.0,2423.03,1.70,0.0,0.0,0.01,91.24,94.14,3.57,0.0,0.0
2019-12-26 16:01:14,9.0,5.57,0.04,0.0,2423.03,1.70,0.0,0.0,0.01,91.24,91.78,5.93,0.0,0.0
2019-12-26 16:01:24,9.0,5.57,0.04,0.0,2423.03,1.70,0.0,0.0,0.01,91.27,84.70,13.01,0.0,0.0
2019-12-26 16:01:34,9.0,5.77,0.04,0.0,2423.03,1.70,0.0,0.0,0.01,91.27,79.24,18.47,0.0,0.0
2019-12-26 16:01:44,9.0,6.41,0.04,0.0,2423.03,1.70,0.0,0.0,0.01,91.24,77.86,19.85,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-16 18:44:04,13.0,-0.35,0.00,0.0,3623.66,12.39,0.0,0.0,0.00,48.70,2241.00,9.26,0.0,0.0
2020-03-16 18:44:14,13.0,-0.35,0.00,0.0,3623.66,12.39,0.0,0.0,0.00,48.70,2241.00,9.26,0.0,0.0
2020-03-16 18:44:24,13.0,-0.35,0.00,0.0,3623.66,12.39,0.0,0.0,0.00,48.70,2241.00,9.26,0.0,0.0
2020-03-16 18:44:34,13.0,-0.35,0.00,0.0,3623.66,12.39,0.0,0.0,0.00,48.70,2241.00,9.26,0.0,0.0


### Создание таблички param_limits

In [32]:
name = []
min_val = []
max_val = []

# Заполняем списки данными
for col in df_well_ch.columns:
    name.append(col)  # Имя столбца
    min_val.append(df_well_ch[col].min())  # Минимальное значение
    max_val.append(df_well_ch[col].max())  # Максимальное значение

# Создаем новый DataFrame
df = pd.DataFrame({
    'name': name,
    'min_val': min_val,
    'max_val': max_val
})

# Выводим новый DataFrame
print(df)


                      name  min_val   max_val
0           Операции (euc)     1.00     17.00
1         Вес на крюке (t)    -0.45    129.75
2                  Газ (%)    -0.02     11.96
3           Давление (kpa)     0.00  26035.18
4                Забой (m)   980.21   3857.39
5      Мех. скорость (m/h)     0.21   1344.00
6             Момент (knm)     0.00      0.03
7   Нагрузка на долото (t)     0.00    439.71
8     Обороты ротора (rpm)     0.00    113.81
9      Объем раствора (m3)     3.03    136.09
10    Положение долота (m)   -10.33   3857.39
11            Тальблок (m)    -9.49     25.61
12     Ходы насоса 1 (spm)     0.00    151.00
13     Ходы насоса 2 (spm)     0.00    150.00


In [33]:
df['min_val'] = 0
df = df.drop(index=[0, 4, 5, 10, 11, 12, 13])


In [35]:
df.reset_index(drop=True, inplace=True)
df

,name,min_val,max_val
0,Вес на крюке (t),0,129.75
1,Газ (%),0,11.96
2,Давление (kpa),0,26035.18
3,Момент (knm),0,0.03
4,Нагрузка на долото (t),0,439.71
5,Обороты ротора (rpm),0,113.81
6,Объем раствора (m3),0,136.09


In [38]:
# Создаю табличку с максимумами и минимумами нужных параметров в соответствие с размерностью(создавала по табличке с прошлого семестра)

df.loc[df['name'] == 'Вес на крюке (t)', 'max_val'] = 300
df.loc[df['name'] == 'Газ (%)', 'max_val'] = 100
df.loc[df['name'] == 'Давление (kpa)', 'max_val'] = 35463.75 
df.loc[df['name'] == 'Момент (knm)', 'max_val'] = 140 
df.loc[df['name'] == 'Нагрузка на долото (t)', 'max_val'] = 25 
df.loc[df['name'] == 'Обороты ротора (rpm)', 'max_val'] = 200 
df.loc[df['name'] == 'Объем раствора (m3)', 'max_val'] = 240 # взяла за параметр volume in tanks TVT
#df.loc[df['name'] == 'Забой (m)', 'max_val'] = 
#df.loc[df['name'] == 'Тальблок (m)', 'max_val'] = 
df

,name,min_val,max_val
0,Вес на крюке (t),0,300.00
1,Газ (%),0,100.00
2,Давление (kpa),0,35463.75
3,Момент (knm),0,140.00
4,Нагрузка на долото (t),0,25.00
5,Обороты ротора (rpm),0,200.00
6,Объем раствора (m3),0,240.00


In [39]:
df.to_csv('param_limits.csv')

In [40]:
lims = pd.read_csv('param_limits.csv')
lims

,Unnamed: 0,name,min_val,max_val
0,0,Вес на крюке (t),0,300.00
1,1,Газ (%),0,100.00
2,2,Давление (kpa),0,35463.75
3,3,Момент (knm),0,140.00
4,4,Нагрузка на долото (t),0,25.00
5,5,Обороты ротора (rpm),0,200.00
6,6,Объем раствора (m3),0,240.00


### Подсчёт числа операций

In [8]:
start = datetime.datetime.now()
print('Время старта: ' + str(start))

"""
data_path = Path('/s/ls4/users/selishcheva/VKR/well_data/')

for fp in data_path.iterdir():
    df_i = pd.read_csv(f'/s/ls4/users/selishcheva/VKR/well_data/{fp.name}', index_col=0, sep=';')
    
    if 'Операции (euc)' in df_i.columns:
        unique_operations = list(set(df_i['Операции (euc)']))
"""

unique_operations_set = set()
directory_path = Path('/s/ls4/users/selishcheva/VKR/well_data')

for fp in path_to_data.iterdir():
    
    if fp.is_file():
        
        df_i = pd.read_csv(fp, index_col=0, sep=',')
        if 'Операции (euc)' in df_i.columns:
            unique_operations_set.update(df_i['Операции (euc)'].unique())

unique_operations_list = list(unique_operations_set)
print(unique_operations_list)

finish = datetime.datetime.now()
print('Время окончания: ' + str(finish))
print('Время работы: ' + str(finish - start))

Время старта: 2024-10-22 00:39:58.133045
[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0]
Время окончания: 2024-10-22 00:40:13.094635
Время работы: 0:00:14.961590


In [7]:
start = datetime.datetime.now()
print('Время старта: ' + str(start))

path_to_data = Path('/s/ls4/users/selishcheva/VKR/well_data')

count = 0
for fp in path_to_data.iterdir():
    
    if fp.is_file():
        
        df_i = pd.read_csv(fp, index_col=0, sep=',')
        if 'Операции (euc)' not in df_i.columns:
            count +=1
print(count)

finish = datetime.datetime.now()
print('Время окончания: ' + str(finish))
print('Время работы: ' + str(finish - start))

Время старта: 2024-10-22 00:39:28.337126
0
Время окончания: 2024-10-22 00:39:42.716454
Время работы: 0:00:14.379328
